## Build Environment

In [ ]:
# BUILD ENV for a runpod preloaded with PyTorch 1.13.1
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchtext==0.13.1 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install pytorch_lightning==1.6.5

#version shouldn't matter here
!pip install omegaconf
!pip install einops
!pip install test-tube
!pip install huggingface_hub
!pip install gdown
!pip install transformers
!pip install kornia


!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0

#execute setup.py
!pip install -e .

!pip install protobuf==3.20.1
!pip install -qq diffusers["training"]==0.3.0
!pip install ftfy
!pip install captionizer==1.0.1
!pip install taming-transformers-rom1504 
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip





: 

# Download a model from hugging face
You can also provide your own v1.* model for training by uploading it and renaming it to "model.ckpt".  It should be in the same directory as dreambooth.ipynb

In [ ]:
# Download the 1.5 sd model
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

downloaded_model_path = hf_hub_download(
 repo_id="runwayml/stable-diffusion-v1-5",
 filename="v1-5-pruned.ckpt"
)

# Move the downloaded ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

# Download pre-generated regularization images
We've created the following image sets

`man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
`man_unsplash` - pictures from various photographers
`300_person_ddim`
`person_ddim`
`woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
`person_ddim` is recommended

In [ ]:
#Download Regularization Images

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "300_person_ddim", "person_ddim", "woman_ddim", "blonde_woman"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_images
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "project_name"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 2000

# Match class_word to the category of the regularization images you chose above.
class_word = "person" # typical uses are "man", "person", "woman"

# If you are training a person's face, set this to True
i_am_training_a_persons_face = True

flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5

# This is the unique token you are incorporating into the stable diffusion model.
token = "firstNameLastName"

# Default config, saves the checkpoint when max_training_steps is reached.
config = "v1-finetune_unfrozen.yaml"

# Uncomment the following line to save checkpoints every 250 steps.
#config = "v1-finetune_unfrozen_save_checkpoints_every_250_steps.yaml"

# Uncomment the following line to save checkpoints every 500 steps.
#config = "v1-finetune_unfrozen_save_checkpoints_every_500_steps.yaml"

reg_data_root = "/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base "configs/stable-diffusion/{config}" \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/Dreambooth-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test \
 --flip_p {flip_p_arg}

## Copy and name the checkpoint file(s)

In [ ]:
import re

training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"

if config == "v1-finetune_unfrozen.yaml":
 # Copy the checkpoint into our `trained_models` folder
 directory_paths = !ls -d logs/*
 last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
 file_name = date_string[-1] + "_" + \
             project_name + "_" + \
             str(len(training_images)) + "_training_images_" + \
             str(max_training_steps) + "_max_training_steps_" + \
             token + "_token_" + \
             class_word + "_class_word.ckpt"

 file_name = file_name.replace(" ", "_")

 !mkdir -p trained_models
 !mv "{last_checkpoint_file}" "trained_models/{file_name}"

 print("Download your trained model from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")
else:
 directory_paths = !ls -d logs/*
 checkpoints_directory = directory_paths[-1] + "/checkpoints/trainstep_checkpoints"
 file_paths = !ls -d "{checkpoints_directory}"/*

 for i, original_file_name in enumerate(file_paths):
  # Remove the "epoch=000000-step=0000" text
  steps = re.sub(checkpoints_directory + "/epoch=\d{6}-step=0*", "", original_file_name)

  # Remove the .ckpt
  steps = steps.replace(".ckpt", "")

  # Setup the filename
  file_name = date_string[-1] + "_" + \
                  project_name + "_" + \
                  str(len(training_images)) + "_training_images_" + \
                  steps + "_training_steps_" + \
                  token + "_token_" + \
                  class_word + "_class_word.ckpt"

  file_name = file_name.replace(" ", "_")

  # Make the directory and move the files into it.
  !mkdir -p trained_models
  !mv "{original_file_name}" "trained_models/{file_name}"

 print("Download your trained models from the 'trained_models' folder and use in your favorite Stable Diffusion repo!")